# Generating music with Markov Chains using real music as training data

## Import all required libraries

- *`random`*: For generating random choices. It is used to select the next note in the music sequence;


- *`mingus`*: Library for handling musical notation and MIDI files;
- *`Note`*: Represents a musical note;
- *`Bar`:* Classes for creating and managing MIDI bars of music;
- *`Track`*: Classes for creating and managing MIDI tracks of music;
- *`midi_file_out`*: Module for writing MIDI files;


- *`pygame`*: Library for creating games. It is used to play the MIDI file sounds;


- *`time`*: Used to pause execution while the music is playing.

In [1]:
import pretty_midi

from collections import defaultdict

import random

import os

import pygame

import time

pygame 2.6.0 (SDL 2.28.4, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Preprocess Music Data

### Extract notes from a MIDI file

The first step is to extract the notes from a MIDI file using the function *`extract_notes_from_midi()`*.
1. The *`extracted_notes`* list will store all extracted notes from the imported MIDI file;
2. The MIDI file is loaded and stored int *`midi_data`*;
3. All the instrument data is extracted by iterating through the MIDI file data. The extracted notes are appended to the *`extracted_notes`* list;
4. The function returns the *`extracted_notes`* list, containing all the extracted data from the MIDI file.

In [2]:
def extract_notes_from_midi(midi_file_path):
    extracted_notes = []
    
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)
    
    # Extract notes from all instruments
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            extracted_notes.append(pretty_midi.note_number_to_name(note.pitch))
    
    return extracted_notes

In [3]:
# MIDI file path
midi_path = "midi_files/Never-Gonna-Give-You-Up-2.mid"

# Calling the extract_notes_from_midi() function and storing all the extracted notes
notes = extract_notes_from_midi(midi_path)

## Build Transformation Matrix

In [4]:
def build_transition_matrix(music_notes):
    transition_matrix_new = defaultdict(list)
    
    for i in range(len(music_notes) - 1):
        current_note = music_notes[i]
        next_note = music_notes[i + 1]
        
        transition_matrix_new[current_note].append(next_note)
    
    return transition_matrix_new

In [5]:
transition_matrix = build_transition_matrix(notes)

## Generate Music

In [6]:
def generate_music(matrix, start_note, length=50):
    music = [start_note]
    current_note = start_note
    
    for _ in range(length - 1):
        next_note = random.choice(matrix[current_note])
        music.append(next_note)
        current_note = next_note
    
    return music

In [7]:
starting_note = random.choice(notes)
generated_music = generate_music(transition_matrix, starting_note, length=20)

## Save Generated Music

In [8]:
def save_generated_music_as_midi(gen_music, midi_save_path):
    midi_data = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # program=0 is Acoustic Grand Piano
    
    start_time = 0.0
    duration = 0.5
    
    for note_name in gen_music:
        note_number = pretty_midi.note_name_to_number(note_name)
        note = pretty_midi.Note(velocity=50,   # Volume Level
                                pitch=note_number, 
                                start=start_time, 
                                end=start_time + duration)
        instrument.notes.append(note)
        
        start_time += duration
    
    midi_data.instruments.append(instrument)
    midi_data.write(midi_save_path)

In [9]:
# Set folder to save MIDI file in
folder_path = "midi_files_generated"

# Check if the provided folder exists, and if it doesn't - create it
os.makedirs(folder_path, exist_ok=True)

# Set MIDI file name
midi_filename = "markov_music_inspired.mid"

output_midi_file_path = os.path.join(folder_path, midi_filename)

save_music = save_generated_music_as_midi(generated_music, output_midi_file_path)

## Play Generated Music

In [10]:
def play_midi_file(midi_file):
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(midi_file)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        time.sleep(1)

    pygame.quit()


In [11]:
# Play the generated MIDI file
play_midi_file(output_midi_file_path)

## Credits

1. MIDI Files Downloaded from [BitMidi](https://bitmidi.com)